In [ ]:
import numpy as np


In [ ]:
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

In [ ]:
print(datain)
print(datain.shape)   # Useful if it’s an array
print(type(datain)) 

In [ ]:
print(dataout)
print(dataout.shape)   # Useful if it’s an array
print(type(dataout)) 

# Week 13
- This is clearly inside the “good basin”
- But it is worse than the best-known configuration
- This indicates we’ve passed the center of the basin slightly in at least one direction

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from scipy.optimize import minimize

# --- Initial data ---
X_init = np.array([
    [0.17152521, 0.34391687, 0.2487372 ],
    [0.24211446, 0.64407427, 0.27243281],
    [0.53490572, 0.39850092, 0.17338873],
    [0.49258141, 0.61159319, 0.34017639],
    [0.13462167, 0.21991724, 0.45820622],
    [0.34552327, 0.94135983, 0.26936348],
    [0.15183663, 0.43999062, 0.99088187],
    [0.64550284, 0.39714294, 0.91977134],
    [0.74691195, 0.28419631, 0.22629985],
    [0.17047699, 0.6970324 , 0.14916943],
    [0.22054934, 0.29782524, 0.34355534],
    [0.66601366, 0.67198515, 0.2462953 ],
    [0.04680895, 0.23136024, 0.77061759],
    [0.60009728, 0.72513573, 0.06608864],
    [0.96599485, 0.86111969, 0.56682913]
])

y_init = np.array([
    -0.1121222 , -0.08796286, -0.11141465, -0.03483531, -0.04800758,
    -0.11062091, -0.39892551, -0.11386851, -0.13146061, -0.09418956,
    -0.04694741, -0.10596504, -0.11804826, -0.03637783, -0.05675837
])

# --- Add new observations from previous rounds ---
X_new = np.array([
    [0.901884, 0.273619, 0.723590],
    [0.393393, 0.209704, 0.929527],
    [0.117994, 0.128418, 0.478883],
    [0.442975, 0.132914, 0.276531],
    [0.683163, 0.340182, 0.318758],
    [0.514909, 0.429977, 0.441068],
    [0.376435, 0.333454, 0.472888],
    [0.597924, 0.348150, 0.493042],
    [0.484909, 0.399977, 0.441068],
    [0.599163, 0.483661, 0.474838],
    [0.579367, 0.438291, 0.422150],
    [0.509607, 0.487857, 0.484160]
])

y_new = np.array([
    -0.13917950276182944,
    -0.1691006942213739,
    -0.043036710546582216,
    -0.11152705861024295,
    -0.05490061720239002, 
    -0.0050467853497373665,
    -0.017389844227996235,
    -0.009890240573922564,
    -0.02634297118984294,
    -0.010299714267429757,
    -0.014669706855365663,
    -0.015233202837457842
])

# Combine all data
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])


The landscape you’ve observed tells us:
- The basin is narrow
- Sensitivity is highest in x₂ and x₃
- Large moves will only make things worse

So we’ll:
- Stay very close
- Bias toward exploitation
- Allow just enough uncertainty to correct small GP bias

Strategy:
- Radius: ±0.01 (max ±0.015 if you want to be slightly bolder)
- EI with very small xi → exploitation-heavy
- Constrained local bounds → no wandering

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from scipy.optimize import minimize

# Best point so far
x_best = np.array([0.514909, 0.429977, 0.441068])

# Tight local bounds (clip to [0,1])
delta = 0.01
bounds = [
    (max(0, x_best[0] - delta), min(1, x_best[0] + delta)),
    (max(0, x_best[1] - delta), min(1, x_best[1] + delta)),
    (max(0, x_best[2] - delta), min(1, x_best[2] + delta)),
]

# GP (keep it tight and stable)
kernel = Matern(length_scale=0.2, length_scale_bounds=(0.05, 0.5), nu=2.5)
gp = GaussianProcessRegressor(
    kernel=kernel,
    alpha=1e-4,
    normalize_y=True,
    n_restarts_optimizer=10,
    random_state=0
)

gp.fit(X_all, y_all)
y_best = y_all.max()

def expected_improvement(x, gp, y_best, xi=0.001):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = float(mu[0]), float(sigma[0])
    if sigma < 1e-12:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    return imp * norm.cdf(Z) + sigma * norm.pdf(Z)

# Optimize locally
best_x, best_ei = None, -np.inf
for _ in range(50):
    x0 = np.array([
        np.random.uniform(b[0], b[1]) for b in bounds
    ])
    res = minimize(
        lambda x: -expected_improvement(x, gp, y_best),
        x0=x0,
        bounds=bounds,
        method="L-BFGS-B"
    )
    if res.success:
        ei = -res.fun
        if ei > best_ei:
            best_ei = ei
            best_x = res.x

mu, sigma = gp.predict(best_x.reshape(1,-1), return_std=True)

print("Final refinement point:", np.round(best_x, 6))
print("Predicted mu:", mu[0])
print("Predicted sigma:", sigma[0])
print("EI:", best_ei)
print("Kernel:", gp.kernel_)


# Final Result
- Best so far [0.514909, 0.429977, 0.441068] →  -0.00505   
- Local proble: [0.524909, 0.439977, 0.451068] →  -0.01285
- Exactly the kind of small, symmetric perturbation we expect to test basin shape
- this is sitting near a local minimum basin
- This point confirms rather than contradicts
- [0.514909, 0.429977, 0.441068] is very likely the best attainable point within the budget and model assumptions.